# Datawhale 零基础入门数据挖掘-Task5 模型融合

## 五、模型融合

## 1.模型融合简介

模型融合是在已有多个训练结果的基础上，对多个训练结果进行学习，得到一个更好的学习结果，属于集成学习的方法。目前常用的模型集成方法除了算数平均，几何平均等方法之外有四种，分别是：

* Stacking

* Blending

* bagging

* boosting

集成学习是一种机器学习范式。在集成学习中，我们会训练多个模型（通常称为「弱学习器」）解决相同的问题，并将它们结合起来以获得更好的结果。最重要的假设是：当弱模型被正确组合时，我们可以得到更精确和/或更鲁棒的模型。

在集成学习理论中，我们将弱学习器（或基础模型）称为「模型」，这些模型可用作设计更复杂模型的构件。在大多数情况下，这些基本模型本身的性能并不是非常好，这要么是因为它们具有较高的偏置（例如，低自由度模型），要么是因为他们的方差太大导致鲁棒性不强（例如，高自由度模型）。

集成方法的思想是通过将这些弱学习器的偏置和/或方差结合起来，从而创建一个「强学习器」（或「集成模型」），从而获得更好的性能。

## 2.常用模型集成方法介绍
### 2.1 Stacking

stacking 的概念是学习几个不同的弱学习器，并通过训练一个元模型来组合它们，然后基于这些弱模型返回的多个预测结果输出最终的预测结果。

因此，为了构建 stacking 模型，我们需要定义两个东西：想要拟合的 L 个学习器以及组合它们的元模型。

例如，对于分类问题来说，我们可以选择 KNN 分类器、logistic 回归和SVM 作为弱学习器，并决定学习神经网络作为元模型。然后，神经网络将会把三个弱学习器的输出作为输入，并返回基于该输入的最终预测。

Stacking流程比较复杂因为涉及到交叉验证的过程，其流程与Blending类似，具体如下：

将数据划分为训练集和测试集(test_set)，对训练集进行划分为K个大小相似的集合，取其中一份作为验证集val_set,其余的为训练集train_set；

* 创建第一层的多个模型，这些模型可以使同质的也可以是异质的；
      
* 对于每一个模型来说，train_set和val_set是不一样的；然后利用各自的train_set训练各自的模型，训练好的模型对各自的val_set和test_set进行预测，得到val_predict和test_predict；
    
* 创建第二层的模型，将每个模型对应的val_predict拼接起来作为第二层的训练集，将所有模型的test_predict取平均值作为第二层的测试集；用训练好的第二层模型对第二层的测试集进行预测，得到的结果即为整个测试集的结果
    
————————————————
版权声明：本文为CSDN博主「非典型废言」的原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/sinat_35821976/java/article/details/83622594

### 2.2 Blending
Blending与Stacking大致相同，只是Blending的主要区别在于训练集不是通过K-Fold的CV策略来获得预测值从而生成第二阶段模型的特征，而是建立一个Holdout集。简单来说，Blending直接用不相交的数据集用于不同层的训练。

Blending的流程相较于Stacking来说要简单一些，其流程大致分为以下几步：

* 将数据划分为训练集和测试集(test_set)，其中训练集需要再次划分为训练集(train_set)和验证集(val_set)；

* 创建第一层的多个模型，这些模型可以使同质的也可以是异质的；
    
* 使用train_set训练步骤2中的多个模型，然后用训练好的模型预测val_set和test_set得到val_predict, test_predict1；
    
* 创建第二层的模型,使用val_predict作为训练集训练第二层的模型；
    
* 使用第二层训练好的模型对第二层测试集test_predict1进行预测，该结果为整个测试集的结果
    
————————————————
版权声明：本文为CSDN博主「非典型废言」的原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/sinat_35821976/java/article/details/83622594

### 2.3 begging
该方法通常考虑的是同质弱学习器，相互独立地并行学习这些弱学习器，并按照某种确定性的平均过程将它们组合起来。
begging算法使用之前，需要明确该算法的两个假设：

首先初始数据集的大小 N 应该足够大，以捕获底层分布的大部分复杂性。这样，从数据集中抽样就是从真实分布中抽样的良好近似（代表性）。

其次，与自助样本的大小 B 相比，数据集的规模 N 应该足够大，这样样本之间就不会有太大的相关性（独立性）。

其算法过程如下：
* 从原始样本集中抽取训练集。每轮从原始样本集中使用Bootstraping的方法抽取n个训练样本（在训练集中，有些样本可能被多次抽取到，而有些样本可能一次都没有被抽中）。共进行k轮抽取，得到k个训练集。（k个训练集之间是相互独立的）
* 每次使用一个训练集得到一个模型，k个训练集共得到k个模型。（注：这里并没有具体的分类算法或回归方法，我们可以根据具体问题采用不同的分类或回归方法，如决策树、感知器等）
* 对分类问题：将上步得到的k个模型采用投票的方式得到分类结果；对回归问题，计算上述模型的均值作为最后的结果。（所有模型的重要性相同）

### 2.4 boosting
其思想是「迭代地」拟合模型，使模型在给定步骤上的训练依赖于之前的步骤上拟合的模型。

具体思路有两条：

* 通过提高那些在前一轮被弱分类器分错样例的权值，减小前一轮分对样例的权值，来使得分类器对误分的数据有较好的效果。

* 通过加法模型将弱分类器进行线性组合，比如AdaBoost通过加权多数表决的方式，即增大错误率小的分类器的权值，同时减小错误率较大的分类器的权值。而提升树通过拟合残差的方式逐步减小残差，将每一步生成的模型叠加得到最终模型。

## 3. 代码演示
### 3.1 volting、算数平均融合与几何平均融合

In [4]:
def volting_for_classify(y_hat,weight=None):
    #输入y_hat为弱分类器分类结果，格式为list
    assert (len(y_hat)==len(weight))or(weight==None) #权重序列应与预测数据长度相同，或默认为1
    class_list = []
    count_list = []
    cal_weight = []
    if weight==None:
        for i in range(len(y_hat)):
            cal_weight.append(1)
    else:
        cal_weight = weight
    for i in range(len(y_hat)):
        add_class = True
        for j in range(len(class_list)):
            if y_hat[i] ==  class_list[j]:
                count_list[j] += cal_weight[i]
                add_class = False
                break
        if add_class:
            class_list.append(y_hat[i])
            count_list.append(cal_weight[i])
    return class_list[count_list.index(max(count_list))]

def cal_mean_for_predict(y_hat,power=1):
    #power为算数平均幂次，当默认为1，即普通平均
    assert power>=1
    y = 0
    for i in range(len(y_hat)):
        y += y_hat[i]**power
    y = y/len(y_hat)
    y = y**(1.0/power)
    return y

def geometric_Mean_for_predict(y_hat):
    y = 1
    for i in range(len(y_hat)):
        y *= y_hat[i]
    y = y**(1.0/len(y_hat))
    return y

### 3.2 Stacking

In [6]:
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

def stacking_for_predict(train_reg1,train_reg2,train_reg3,y_train_true,test_pre1,test_pre2,test_pre3,model_L2= linear_model.LinearRegression()):
    model_L2.fit(pd.concat([pd.Series(train_reg1),pd.Series(train_reg2),pd.Series(train_reg3)],axis=1).values,y_train_true)
    Stacking_result = model_L2.predict(pd.concat([pd.Series(test_pre1),pd.Series(test_pre2),pd.Series(test_pre3)],axis=1).values)
    return Stacking_result

def stacking_for_classify(model,meta_model=LogisticRegression()):
    sclf = StackingClassifier(classifiers=model,meta_classifier=meta_model)

### 3.3 Blending

In [9]:
from sklearn.datasets import make_blobs
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

def blending_for_classify(data,target,model):
    X, X_predict, y, y_predict = train_test_split(data, target, test_size=0.3, random_state=2020)
    dataset_blend_train = np.zeros((data.shape[0], len(model)))
    dataset_blend_test = np.zeros((X_predict.shape[0], len(model)))

    n_splits = 5
    skf = StratifiedKFold(n_splits)
    skf = skf.split(X, y)

    for j, clf in enumerate(clfs):
        dataset_blend_test_j = np.zeros((X_predict.shape[0], len(model)))
        for i, (train, test) in enumerate(skf):
            X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
            clf.fit(X_train, y_train)
            y_submission = clf.predict_proba(X_test)[:, 1]
            dataset_blend_train[test, j] = y_submission
            dataset_blend_test_j[:, i] = clf.predict_proba(X_predict)[:, 1]
        dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)

    clf = LogisticRegression(solver='lbfgs')
    clf.fit(dataset_blend_train, y)
    y_submission = clf.predict_proba(dataset_blend_test)[:, 1]
    return y_submission

### 3.4 Begging

In [14]:
import copy

def begging(train_data, test_data, model, sample_rate = 0.5, sample_round = 10):
    index = np.random.randint(0,train_data.shape[0],(int(train_data.shape[0]*rate),sample_round))
    train_sample = []
    for i in range(len(index)):
        train_sample.append(train_data.iloc[index])
    predict_data = []
    for i in range(len(train_sample)):
        temp_model = copy.copy(model.fit(train_sample[i]))
        predict_data.append(temp_model.predict(test_data))
    return predict

def begging_for_predict(train, test, model, sample = 0.5, rd = 10):
    pred = begging(train,test,model,sample,rd)
    return cal_mean_for_predict(pred)

### 3.5 Boosting
由于boosting本身未给定权重计算方法，故由这种方法分类得到不同的算法，例如Adaboosting与Gradient Boosting，且被许多算法集成，例如LGBM，这里不做演示。

## 4.应用集成的常见决策树算法
* Bagging + 决策树 = 随机森林
* AdaBoost + 决策树 = 提升树
* Gradient Boosting + 决策树 = GBDT

## 5.集成算法对比

### 5.1 Bagging和Boosting的区别：
1）样本选择上：

Bagging：训练集是在原始集中有放回选取的，从原始集中选出的各轮训练集之间是独立的。

Boosting：每一轮的训练集不变，只是训练集中每个样例在分类器中的权重发生变化。而权值是根据上一轮的分类结果进行调整。

2）样例权重：

Bagging：使用均匀取样，每个样例的权重相等

Boosting：根据错误率不断调整样例的权值，错误率越大则权重越大。

3）预测函数：

Bagging：所有预测函数的权重相等。

Boosting：每个弱分类器都有相应的权重，对于分类误差小的分类器会有更大的权重。

4）并行计算：

Bagging：各个预测函数可以并行生成

Boosting：各个预测函数只能顺序生成，因为后一个模型参数需要前一轮模型的结果。


### 5.2 Stacking和Blending的区别：


* Blending的优点在于使用比Stacking简单，训练速度比Stacking快（使用了较少的数据），避免了信息泄露（留出的测试集未被任何模型用于训练）。

* 而缺点在于使用了很少的数据（由于额外留出了一部分测试集，所以使用数据较少），由于较少的数据涵盖信息完整程度较低，在同样精度下Blending的训练精度更可能是因为过拟合导致的。

### 5.3 基础模型要求区别
* Stacking：模型可以同质也可异质，一般来说同质模型由于融合方法问题，效果不如Boosting，故一般用于融合异质模型。其本质是对于已有模型结果重新进行学习的一个误差学习修正方法。
* Blending：模型可以同质也可异质，一般来说同质模型由于融合方法问题，效果不如Boosting，故一般用于融合异质模型。
* Begging：模型要求同质，由于算法本身是对训练集做出处理，在多次抽样中提取相近特征，故要求不同组模型训练基础相同。
* Boosting：模型可以同质也可异质，其实质是将多个分类器按一定权重融合，求权重的过程，故只要求结果。

## 6.总结

模型融合是在已有多个训练结果的基础上，对多个训练结果进行学习，得到一个更好的学习结果，属于集成学习的方法。

依据开源学习笔记，包括如下几个层面的融合

1）结果层面的融合（加权，投票），这种是最常见的融合方法，其可行的融合方法也有很多，比如根据结果的得分进行加权融合，还可以做Log，exp处理等。在做结果融合的时候，有一个很重要的条件是模型结果的得分要比较近似，然后结果的差异要比较大，这样的结果融合往往有比较好的效果提升。

2）特征层面的融合（Begging），这个层面其实感觉不叫融合，准确说可以叫分割，很多时候如果我们用同种模型训练，可以把特征进行切分给不同的模型，然后在后面进行模型或者结果融合有时也能产生比较好的效果。

3）模型层面的融合（Stacking），模型层面的融合可能就涉及模型的堆叠和设计，比如加Staking层，部分模型的结果作为特征输入等，这些就需要多实验和思考了，基于模型层面的融合最好不同模型类型要有一定的差异，用同种模型不同的参数的收益一般是比较小的。

## 7.参考资料
* Datawhale开源学习笔记

* https://www.jianshu.com/p/369e3f4d239f

* https://www.cnblogs.com/liuwu265/p/4690486.html

* https://blog.csdn.net/maqunfi/article/details/82220115

* https://baijiahao.baidu.com/s?id=1633580172255481867&wfr=spider&for=pc

* https://www.jianshu.com/p/59313f43916f

* https://blog.csdn.net/sinat_35821976/article/details/83622594

* https://zhuanlan.zhihu.com/p/42229791